In [5]:
from qutip import basis, sigmax, sigmaz, sigmay, Options 
from qutip.solver.heom import DrudeLorentzBath
from qutip.solver.heom import HEOMSolver
from qutip.solver.heom import DrudeLorentzPadeBath
from qutip.solver.heom import BathExponent
import math
__all__ = ['parallel_map', 'serial_map', 'loky_pmap', 'mpi_pmap']

import multiprocessing
import os
import sys
import time
import threading
import concurrent.futures
import warnings
from qutip.ui.progressbar import progress_bars
from qutip.settings import available_cpu_count
from scipy.fft import fft, ifft 
from scipy.special import shichi
from mpmath import meijerg

import matplotlib.pyplot as plt
import qutip

import numpy as np 
import time 




# More flexibel solver for custom bath
from qutip.solver.heom import HSolverDL
from qutip.solver.heom import BosonicBath

import qutip.solver.brmesolve as brmesolve

In [6]:
"""Define Constants"""
#hbar, kb=1
we=1 # With hbar = 1 this is the upper energy
wg=0# ground state
ve=0.1 #  ground state frequencie from the exited atomic state in 0.1*we 
vg=0.1 # ground state frequencie from the ground atomic state 
λ1=0.1# dim less coupling constant depending on the shift of the energy minima shifts in space
λ2=vg**2-ve**2/4*vg**2 # dim less coupling constant depending on the vibronic freq 
w0=we- wg + λ1**2 *vg**3 /ve**2
w00 = we - wg + (ve - vg )/2
g1=0.1 #couplig to the atom to the bath   
g2=0.1 #couplig to the vibronic modes to the bath 
gamma_1=0.1
gamma_2=0.1
beta=10 #temp Einheit von  hb*omega durch kb
#nb=1/(np.exp((w) * beta)-1)
nb=1
N=2# size of the vibronic hilbi space 
#tlist=np.linspace[0,1,10]
# operators
a  = qutip.tensor(qutip.qeye(2),qutip.destroy(N)) #ladder operator of the vibronic modes 
P12 = qutip.tensor(basis(2,0)*basis(2,1).dag(),qutip.qeye(N))
P21 = qutip.tensor(basis(2,1)*basis(2,0).dag(),qutip.qeye(N))
P22 = qutip.tensor(basis(2,0)*basis(2,0).dag(),qutip.qeye(N))
P22 = qutip.tensor(basis(2,1)*basis(2,1).dag(),qutip.qeye(N))
psi0 = qutip.tensor(basis(2,1),basis(N,0))   #|e,0>
rho0=qutip.tensor(basis(2,1)*basis(2,1).dag(),basis(N,0)*basis(N,0).dag()) #|e,0><e,0|


#System hamiltonian
H= vg*a.dag()*a +w0*P21*P12 + λ1*vg*(a+a.dag())*P21*P12+λ2*vg*(a+a.dag())**2  *P21*P12                     
H_diag=vg*P12*P21*a.dag()*a+ (vg*a.dag()*a+w00)*P21*P12
# H_int 
O1=P12+P21
O2=a+a.dag()

evals , ekets = H.eigenstates()
evals2 , ekets2 = H_diag.eigenstates()
print(H,H_diag)
#print(ekets,ekets2)
state1=qutip.tensor( basis(2,1),qutip.basis(N,0))
state2=qutip.tensor(basis(2,0),qutip.basis(N,1))
#print(state1.dag()*H_diag*state2)
def SS(H):
    return (-H*beta).expm()/((-H*beta).expm().tr())
print((SS(H)*P22).tr())
print((SS(H_diag)*P22).tr())

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.        0.        0.        0.       ]
 [0.        0.1       0.        0.       ]
 [0.        0.        1.0019975 0.01     ]
 [0.        0.        0.01      1.1019975]] Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.  0.  0.  0. ]
 [0.  0.1 0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1.1]]
4.470588506642831e-05
4.5397868702434395e-05


In [10]:
# decay 
print(a*P12*P21*P21*P12*a.dag())
########################################################################################################
gamma_1=0.1
gamma_2=0.1
c_op_list=[]

c_op_list.append(np.sqrt((nb+1)*gamma_1)*P12)
c_op_list.append(np.sqrt((nb)*gamma_1)*P21)
c_op_list.append(np.sqrt((nb+1)*gamma_1)*a)
c_op_list.append(np.sqrt((nb)*gamma_1)*a.dag())
#Resul_Lindblad=qutip.mesolve(H, rho0, tlist, c_ops=c_op_list, P22, args={}, options=None, progress_bar=None, _safe_mode=True)

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
